# Recommending Music to train to using an existing Spotify playlist

*Code by Sean Brockway*

### Necessary imports for Spotify and MongoDB Connections

In [24]:
import requests
import json
import pymongo
import spotipy
import spotipy.util as util 
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from spotipy.oauth2 import SpotifyClientCredentials
from pymongo import MongoClient
import urllib.parse

### Setting up connections for MongoDB and Spotify API

In [37]:
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

conn = MongoClient("mongodb://localhost:27017/")
db = conn['gym-music-database']
test_playlist_col = db['Charlie Gym Playlist']
playlist_songs = test_playlist_col.find({})

**Verifying our data is correct, this is an example of a song in our playlist**

In [38]:
print(playlist_songs[0])

{'_id': ObjectId('5ea9ac909301f61e040c13d7'), 'track': 'Machu Picchu', 'track_id': '6QNnvkgk4DmRzosGBODltZ', 'artist_name': ['The Strokes'], 'artist_id': ['0epOFNiUfyON9EYx7Tpr6V'], 'genres': ['alternative rock', 'garage rock', 'modern rock', 'permanent wave', 'rock'], 'danceability': 0.69, 'energy': 0.821, 'key': 1, 'loudness': -5.282, 'mode': 1, 'speechiness': 0.0639, 'acousticness': 0.0128, 'instrumentalness': 0.159, 'liveness': 0.0545, 'valence': 0.867, 'tempo': 105.007, 'duration_ms': 209464, 'time_signature': 4}


**Next we need to make a list of every song's genres**

In [39]:
genre_list = []
for song in playlist_songs:
    genre_list.extend(song['genres'])
genre_list

['alternative rock',
 'garage rock',
 'modern rock',
 'permanent wave',
 'rock',
 'alternative rock',
 'dance-punk',
 'garage rock',
 'modern rock',
 'new rave',
 'rock',
 'swedish garage rock',
 'swedish hard rock',
 'swedish indie rock',
 'modern rock',
 'rock',
 'alternative rock',
 'garage rock',
 'modern rock',
 'permanent wave',
 'rock',
 'alternative rock',
 'garage rock',
 'modern rock',
 'permanent wave',
 'rock',
 'alternative rock',
 'britpop',
 'garage rock',
 'indie rock',
 'leicester indie',
 'modern rock',
 'rock',
 'garage rock',
 'modern rock',
 'permanent wave',
 'rock',
 'sheffield indie',
 'alternative rock',
 'britpop',
 'garage rock',
 'indie rock',
 'leicester indie',
 'modern rock',
 'rock',
 'alternative rock',
 'britpop',
 'garage rock',
 'indie rock',
 'leicester indie',
 'modern rock',
 'rock',
 'alternative rock',
 'britpop',
 'garage rock',
 'indie rock',
 'leicester indie',
 'modern rock',
 'rock',
 'alternative rock',
 'garage rock',
 'modern rock',
 'po

**Next we need to find the most common occuring genres to base our recommendations on**

In [40]:
from collections import Counter
most_common_genres = [genre for genre, genre_count in Counter(genre_list).most_common(8)]
print(most_common_genres)

['modern rock', 'rock', 'garage rock', 'alternative rock', 'indie pop', 'permanent wave', 'indie rock', 'dance-punk']


To get ideas for songs to recommend, the search end point and recommendations end point in the API can be used. For the search end point, spaces will not work, so it is best to use the "replace" method to change spaces to "%20"

The recommendations end point in the Spotify API can also be used to generate a list of song recommendations, this does not require any text to be replaced. 

In [41]:
search_genres = []
for genre in most_common_genres:
    search = genre.replace(" ", "%20")
    search_genres.append(search)
print(search_genres)

['modern%20rock', 'rock', 'garage%20rock', 'alternative%20rock', 'indie%20pop', 'permanent%20wave', 'indie%20rock', 'dance-punk']


#### Now find recommendations using the 5 most common genres. 
It is worth noting that the recommendations end point has lots of different paramaters and kwargs, and bespoke recommendations can be made using the detailed spotify audio features. For now though, we will just use genre.

Firstly we need to get all the available genre seeds used to generate recommendations

In [42]:
seeds = sp.recommendation_genre_seeds()
print(seeds['genres'])

['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep', 'power-po

Now we find the genres in our playlist that can be used as a genre seed

In [48]:
list_A = most_common_genres
list_B = seeds['genres']
jVal = "|".join(list_A)

matches = [i for i in list_B if i in jVal ]
truncated_matches = matches[:-2]
print(truncated_matches)

['alternative', 'dance', 'garage', 'indie', 'pop']


In [49]:
query_string = ','.join(truncated_matches)
print(query_string)

alternative,dance,garage,indie,pop


In [51]:
recommended_tracks = sp.recommendations(seed_genres = truncated_matches, limit=20)
print(recommended_tracks['tracks'])

[{'album': {'album_type': 'ALBUM', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/20JZFwl6HVl6yg8a4H3ZqK'}, 'href': 'https://api.spotify.com/v1/artists/20JZFwl6HVl6yg8a4H3ZqK', 'id': '20JZFwl6HVl6yg8a4H3ZqK', 'name': 'Panic! At The Disco', 'type': 'artist', 'uri': 'spotify:artist:20JZFwl6HVl6yg8a4H3ZqK'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'SA', 'SE', 'SG', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'US', 'UY', 'VN', 'ZA'], 'external_urls': {'spotify': 'https://open.spotify.com/album/1hxraaWEf3wFnJxADf8Dge'}, 'href': 'https://api.spotify.com/v1/albums/1hxraaWEf3wFnJxADf8Dge', 'id': '1hxra

Here are the recommended tracks from Spotify, now we need to make sure there are no duplicate tracks that exist in the existing playlist, then we need to get the audio features for each track and prepare them for our classifier. 

In [53]:
for track  in recommended_tracks['tracks']:
    if any(track['id'] in songs for songs in playlist_songs):
        print('match found')
    else:
        print('no match')

no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match


No matches have been found, now we can begin getting the audio features for each song and building a data frame for our model to make predictions on. 

In [68]:
dataframe_columns = ['track', 'track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
song_list = []
for track in recommended_tracks['tracks']:
    features = sp.audio_features(track['id'])
    if (features[0] is not None):
                song_record = { 
                    "track": track['name'], 
                    "track_id": track['id'],
                    "danceability" : features[0]['danceability'],
                    "energy" : features[0]['energy'],
                    "key" : features[0]['key'],
                    "loudness" : features[0]['loudness'],
                    "mode" : features[0]['mode'],
                    "speechiness" : features[0]['speechiness'],
                    "acousticness" : features[0]['acousticness'],
                    "instrumentalness" : features[0]['instrumentalness'],
                    "liveness" : features[0]['liveness'],
                    "valence" : features[0]['valence'],
                    "tempo" : features[0]['tempo'],
                    "duration_ms" : features[0]['duration_ms'],
                    "time_signature" : features[0]['time_signature']
                } 
                song_list.append(song_record)
#print(song_list)
dfRecommendations = pd.DataFrame.from_dict(song_list)
dfRecommendations

,track,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Miss Jackson (feat. LOLO),46Kcradxrva9Dny4lHU1b3,0.642,0.873,2,-5.821,1,0.0551,0.003940,0.000000,0.1750,0.662,82.002,192053,4
1,Hideaway,0c6xIDDpzE81m2q797ordA,0.838,0.720,7,-4.135,0,0.0483,0.086200,0.006970,0.0772,0.204,122.990,251987,4
2,Whippin (feat. Felix Snow),3QWjljChcOMkRDYSzF33Qr,0.870,0.502,5,-7.497,0,0.0549,0.549000,0.003560,0.0916,0.673,82.530,160597,4
3,Misery Business,6SpLc7EXZIPpy0sVko0aoU,0.517,0.906,1,-3.677,1,0.0735,0.002720,0.000009,0.1130,0.731,172.977,211520,4
4,What Do You Mean?,6nRwc5GgNvBMkKaynhQzrm,0.841,0.571,5,-8.066,0,0.1020,0.650000,0.000330,0.0895,0.772,125.032,207547,4
5,Outrageous (feat. Lynn Eden),1orVdJNvZrp1o5rxIM3EHf,0.819,0.847,7,-6.559,0,0.0614,0.015300,0.411000,0.0603,0.973,130.059,255400,4
6,Home,5xgVwIFfrm3yhLByAKzeYs,0.675,0.643,7,-9.943,1,0.0535,0.183000,0.660000,0.1320,0.342,118.964,275958,4
7,Naive,7BHPGtpuuWWsvE7cCaMuEU,0.391,0.808,8,-6.209,0,0.0892,0.075900,0.000000,0.1490,0.737,100.304,203507,4
8,Let Me Live (feat. Anne-Marie & Mr Eazi),3D1rlKmZdpYUeMtRRLNawc,0.662,0.844,8,-5.495,1,0.0669,0.002950,0.000048,0.5250,0.763,145.007,205760,4
9,My Destiny - Wideboys Remix,0M3fmwAXVHt9xzjkn8QuDA,0.679,0.892,1,-8.448,1,0.1370,0.001570,0.029400,0.0301,0.424,125.984,422520,4


Now we have a data frame which we can use to predict if a song is gym suitable or not. Before this happens, we need to train our model. We will do this with 10 thousands records of training data using the random forest algorthim. 

In [55]:
training_col = db['medium_test_songlist']
training_songs = training_col.find({})
dfTraining =  pd.DataFrame(list(training_songs))
dfTraining

,_id,track,track_id,artist_name,artist_id,from_playlist,genres,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,workout_suitable
0,5e7e4fa0b96e22bd0ee61d99,'Till I Collapse,4xkOaSrkexMciUUogZKVTS,"[Eminem, Nate Dogg]","[7dGJo4pcD2V6oG8kP0tJRR, 1Oa0bMld0A3u5OTYfMzp5h]","Gym Motivation : Workout Motivation, Training ...","[detroit hip hop, g funk, hip hop, rap, g funk...",0.548,0.8470,1,...,1,0.1860,0.06220,0.000000,0.0816,0.1000,171.447,297787,4,1
1,5e7e4fa1b96e22bd0ee61d9a,Numb / Encore,5sNESr6pQfIhL3krM8CtZn,"[JAY-Z, Linkin Park]","[3nFkdlSjzX9mRTtwJOzDYB, 6XyY86QOPPrYVGvF9ch6wz]","Gym Motivation : Workout Motivation, Training ...","[east coast hip hop, hip hop, pop rap, rap, al...",0.687,0.7930,2,...,1,0.1660,0.06030,0.000000,0.5820,0.7510,107.045,205733,4,1
2,5e7e4fa1b96e22bd0ee61d9b,Remember the Name (feat. Styles of Beyond),6ndmKwWqMozN2tcZqzCX4K,"[Fort Minor, Styles Of Beyond]","[7dWYWUbO68rXJOcyA7SpJk, 5bf6yYgHODBW5EreBZshpX]","Gym Motivation : Workout Motivation, Training ...","[rap rock, cali rap, rap rock]",0.688,0.8350,8,...,1,0.0911,0.05830,0.000003,0.0795,0.8800,84.858,230493,4,1
3,5e7e4fa1b96e22bd0ee61d9c,In Da Club,4RY96Asd9IefaL3X4LOLZ8,[50 Cent],[3q7HBObVc0L8jNeTe5Gofh],"Gym Motivation : Workout Motivation, Training ...","[east coast hip hop, gangster rap, hip hop, po...",0.902,0.7200,6,...,0,0.3470,0.26000,0.000000,0.0749,0.8050,90.059,193467,4,1
4,5e7e4fa1b96e22bd0ee61d9d,Stronger,4fzsfWzRhPawzqhX8Qt9F3,[Kanye West],[5K4W6rqBFWDnAN6FQUkS6x],"Gym Motivation : Workout Motivation, Training ...","[chicago rap, rap]",0.617,0.7170,10,...,0,0.1530,0.00564,0.000000,0.4080,0.4900,103.992,311867,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12678,5e7e63f38aa9abc22c18141a,Hallelujah,0uCGsNZqjHgiYO4BwN6Cjw,[Ben Laver],[3lTGvG2QAIoGMp7BKeH4C0],Calming Instrumental Covers,[],0.428,0.0415,7,...,0,0.0476,0.98500,0.942000,0.1060,0.2810,71.353,165882,4,0
12679,5e7e63f48aa9abc22c18141b,Close To Me,0kPLGFA5BjDGVkDbu4bLzE,[Mia Carrera],[00zoOzjXViIF4Fs8XQSK9m],Calming Instrumental Covers,"[calming instrumental, piano cover]",0.636,0.2410,4,...,1,0.0494,0.99000,0.902000,0.1510,0.1210,102.094,203587,4,0
12680,5e7e63f48aa9abc22c18141c,Love Me Like You Do,1nyxHNRjeyTlJQwsLMH7C0,[Anna Kitkowska],[45Miu8OyhYvkkQBXhn0MfQ],Calming Instrumental Covers,"[calming instrumental, chill guitar]",0.635,0.2310,11,...,0,0.0419,0.93900,0.947000,0.0925,0.3210,144.229,198208,4,0
12681,5e7e63f48aa9abc22c18141d,Fix You,2ZzqC4NSi374IufPYvGMP6,[Music Lab Collective],[1ylcY77FWeSVQKh5et1VGp],Calming Instrumental Covers,[classify],0.667,0.0498,3,...,1,0.0673,0.99100,0.890000,0.0818,0.0436,125.070,332427,4,0


Here is the dataframe we will use to train our classifier, next we define the features we want the classifier to learn and the feature we want to predict. These are named X and y respectively.

In [56]:
feature_cols = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'mode', 'speechiness', 'tempo', 'time_signature']
X = dfTraining.loc[:, feature_cols]
y = dfTraining.workout_suitable

In [57]:
X

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,mode,speechiness,tempo,time_signature
0,0.06220,0.548,297787,0.8470,0.000000,1,0.0816,1,0.1860,171.447,4
1,0.06030,0.687,205733,0.7930,0.000000,2,0.5820,1,0.1660,107.045,4
2,0.05830,0.688,230493,0.8350,0.000003,8,0.0795,1,0.0911,84.858,4
3,0.26000,0.902,193467,0.7200,0.000000,6,0.0749,0,0.3470,90.059,4
4,0.00564,0.617,311867,0.7170,0.000000,10,0.4080,0,0.1530,103.992,4
...,...,...,...,...,...,...,...,...,...,...,...
12678,0.98500,0.428,165882,0.0415,0.942000,7,0.1060,0,0.0476,71.353,4
12679,0.99000,0.636,203587,0.2410,0.902000,4,0.1510,1,0.0494,102.094,4
12680,0.93900,0.635,198208,0.2310,0.947000,11,0.0925,0,0.0419,144.229,4
12681,0.99100,0.667,332427,0.0498,0.890000,3,0.0818,1,0.0673,125.070,4


In [58]:
y

0        1
1        1
2        1
3        1
4        1
        ..
12678    0
12679    0
12680    0
12681    0
12682    0
Name: workout_suitable, Length: 12683, dtype: int64

## Prediction

#### Constructing a prediction model using X and y

In [59]:
rf = RandomForestClassifier(n_estimators=40)
workout_classifier = rf.fit(X, y)
workout_classifier

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#### Now we prepare the song recommendations to be tested 

In [60]:
X_predict = dfRecommendations.loc[:, feature_cols]
X_predict.shape

(20, 11)

### Using the classifer to predict the recommendations (X_predict)

In [61]:
new_prediction_class = workout_classifier.predict(X_predict)
new_prediction_class

array([1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1])

 This is the array of predictions, we can build a new dataframe to display this information more clearly

In [63]:
dfClassifiedRecommendations = pd.DataFrame({
    'Track Name': dfRecommendations.track,
    'Track_id': dfRecommendations.track_id,
    'Prediction': new_prediction_class
})
dfClassifiedRecommendations

,Track Name,Track_id,Prediction
0,Miss Jackson (feat. LOLO),46Kcradxrva9Dny4lHU1b3,1
1,Hideaway,0c6xIDDpzE81m2q797ordA,1
2,Whippin (feat. Felix Snow),3QWjljChcOMkRDYSzF33Qr,0
3,Misery Business,6SpLc7EXZIPpy0sVko0aoU,1
4,What Do You Mean?,6nRwc5GgNvBMkKaynhQzrm,1
5,Outrageous (feat. Lynn Eden),1orVdJNvZrp1o5rxIM3EHf,1
6,Home,5xgVwIFfrm3yhLByAKzeYs,0
7,Naive,7BHPGtpuuWWsvE7cCaMuEU,1
8,Let Me Live (feat. Anne-Marie & Mr Eazi),3D1rlKmZdpYUeMtRRLNawc,1
9,My Destiny - Wideboys Remix,0M3fmwAXVHt9xzjkn8QuDA,1


Here we have a list of songs with the classifiers prediction of if it is suitable for working out to or not. 
Now all that is left to do is display only the songs predicted as suitable for our hypothetical user.

In [64]:
dict1 = dfClassifiedRecommendations.to_dict('records')
for track in dict1:
    print(track)

{'Track Name': 'Miss Jackson (feat. LOLO)', 'Track_id': '46Kcradxrva9Dny4lHU1b3', 'Prediction': 1}
{'Track Name': 'Hideaway', 'Track_id': '0c6xIDDpzE81m2q797ordA', 'Prediction': 1}
{'Track Name': 'Whippin (feat. Felix Snow)', 'Track_id': '3QWjljChcOMkRDYSzF33Qr', 'Prediction': 0}
{'Track Name': 'Misery Business', 'Track_id': '6SpLc7EXZIPpy0sVko0aoU', 'Prediction': 1}
{'Track Name': 'What Do You Mean?', 'Track_id': '6nRwc5GgNvBMkKaynhQzrm', 'Prediction': 1}
{'Track Name': 'Outrageous (feat. Lynn Eden)', 'Track_id': '1orVdJNvZrp1o5rxIM3EHf', 'Prediction': 1}
{'Track Name': 'Home', 'Track_id': '5xgVwIFfrm3yhLByAKzeYs', 'Prediction': 0}
{'Track Name': 'Naive', 'Track_id': '7BHPGtpuuWWsvE7cCaMuEU', 'Prediction': 1}
{'Track Name': 'Let Me Live (feat. Anne-Marie & Mr Eazi)', 'Track_id': '3D1rlKmZdpYUeMtRRLNawc', 'Prediction': 1}
{'Track Name': 'My Destiny - Wideboys Remix', 'Track_id': '0M3fmwAXVHt9xzjkn8QuDA', 'Prediction': 1}
{'Track Name': "If You're Over Me", 'Track_id': '1oc3jna2ablnfIKL

In [65]:
dictRecommend = []
for track in dict1: 
    if(track['Prediction'] == 1):
        dictRecommend.append(track)
for track in dictRecommend:
    print(track)

{'Track Name': 'Miss Jackson (feat. LOLO)', 'Track_id': '46Kcradxrva9Dny4lHU1b3', 'Prediction': 1}
{'Track Name': 'Hideaway', 'Track_id': '0c6xIDDpzE81m2q797ordA', 'Prediction': 1}
{'Track Name': 'Misery Business', 'Track_id': '6SpLc7EXZIPpy0sVko0aoU', 'Prediction': 1}
{'Track Name': 'What Do You Mean?', 'Track_id': '6nRwc5GgNvBMkKaynhQzrm', 'Prediction': 1}
{'Track Name': 'Outrageous (feat. Lynn Eden)', 'Track_id': '1orVdJNvZrp1o5rxIM3EHf', 'Prediction': 1}
{'Track Name': 'Naive', 'Track_id': '7BHPGtpuuWWsvE7cCaMuEU', 'Prediction': 1}
{'Track Name': 'Let Me Live (feat. Anne-Marie & Mr Eazi)', 'Track_id': '3D1rlKmZdpYUeMtRRLNawc', 'Prediction': 1}
{'Track Name': 'My Destiny - Wideboys Remix', 'Track_id': '0M3fmwAXVHt9xzjkn8QuDA', 'Prediction': 1}
{'Track Name': "If You're Over Me", 'Track_id': '1oc3jna2ablnfIKLYbKujl', 'Prediction': 1}
{'Track Name': 'Chop Suey!', 'Track_id': '5V3mdRI2yQxbSsJGDPc5lD', 'Prediction': 1}
{'Track Name': 'No Good 4 Me (feat. Megaman, Romeo and Lisa Maffia)'

## The recommendations are:

In [66]:
dfResults = pd.DataFrame.from_dict(dictRecommend)
dfResults

,Track Name,Track_id,Prediction
0,Miss Jackson (feat. LOLO),46Kcradxrva9Dny4lHU1b3,1
1,Hideaway,0c6xIDDpzE81m2q797ordA,1
2,Misery Business,6SpLc7EXZIPpy0sVko0aoU,1
3,What Do You Mean?,6nRwc5GgNvBMkKaynhQzrm,1
4,Outrageous (feat. Lynn Eden),1orVdJNvZrp1o5rxIM3EHf,1
5,Naive,7BHPGtpuuWWsvE7cCaMuEU,1
6,Let Me Live (feat. Anne-Marie & Mr Eazi),3D1rlKmZdpYUeMtRRLNawc,1
7,My Destiny - Wideboys Remix,0M3fmwAXVHt9xzjkn8QuDA,1
8,If You're Over Me,1oc3jna2ablnfIKLYbKujl,1
9,Chop Suey!,5V3mdRI2yQxbSsJGDPc5lD,1
